## Workspace Management demo

This notebook presents examples of how to manage and use team workspaces!

In [1]:
import jwt
import requests
import datetime
import boto3
import urllib3
import time
urllib3.disable_warnings()

In [2]:
realm = "eoepca"
base_domain = "apx.develop.eoepca.org"
keycloak_endpoint = f"https://iam-auth.{base_domain}"
workspace_api_endpoint = f'https://workspace-api.{base_domain}/workspaces'
token_endpoint = f"{keycloak_endpoint}/realms/{realm}/protocol/openid-connect/token"
minio_endpoint = "https://minio.develop.eoepca.org"

### token handling

In [3]:
def iam_token(username, password):
    headers = {
        "Cache-Control": "no-cache",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "scope": "roles",
        "grant_type": "password",
        "username": username,
        "password": password,
        "client_id": "demo",
        "client_secret": "demo"
    }    
    response = requests.post(token_endpoint, headers=headers, data=data)
    if response.ok:
        return response.json()["access_token"]
    else:
        print(response)
        return None

In [4]:
#the following example users are expected to exist
token_user1 = iam_token("example-user-1", "changeme")
decoded_token_user1 = jwt.decode(token_user1, options={"verify_signature": False})
print(f"\n{decoded_token_user1}")
token_user2 = iam_token("example-user-2", "changeme")
decoded_token_user2 = jwt.decode(token_user2, options={"verify_signature": False})
print(f"\n{decoded_token_user2}")
token_admin = iam_token("example-admin", "changeme")
decoded_token_admin = jwt.decode(token_admin, options={"verify_signature": False})
print(f"\n{decoded_token_admin}")


{'exp': 1736872227, 'iat': 1736871927, 'jti': '9fb99903-fd72-4545-945e-67fcea50cc09', 'iss': 'https://iam-auth.apx.develop.eoepca.org/realms/eoepca', 'aud': ['ws-example-1736870119', 'eoapi', 'ws-example-1736866203', 'ws-example-1736870026', 'ws-example-79', 'ws-example-78', 'account'], 'sub': '0965f986-de8e-43fa-ba1c-36d520eefe6f', 'typ': 'Bearer', 'azp': 'demo', 'session_state': 'a9c8bb62-023c-4a30-88bd-4c6c0008d565', 'acr': '1', 'realm_access': {'roles': ['offline_access', 'default-roles-eoepca', 'uma_authorization']}, 'resource_access': {'ws-example-1736870119': {'roles': ['ws_access']}, 'eoapi': {'roles': ['stac_editor']}, 'ws-example-1736866203': {'roles': ['ws_access']}, 'ws-example-1736870026': {'roles': ['ws_access']}, 'ws-example-79': {'roles': ['ws_access']}, 'ws-example-78': {'roles': ['ws_access']}, 'account': {'roles': ['manage-account', 'manage-account-links', 'view-profile']}}, 'scope': 'profile email', 'sid': 'a9c8bb62-023c-4a30-88bd-4c6c0008d565', 'email_verified': F

In [5]:
print(decoded_token_user1["aud"])
print(decoded_token_user1["resource_access"])

['ws-example-1736870119', 'eoapi', 'ws-example-1736866203', 'ws-example-1736870026', 'ws-example-79', 'ws-example-78', 'account']
{'ws-example-1736870119': {'roles': ['ws_access']}, 'eoapi': {'roles': ['stac_editor']}, 'ws-example-1736866203': {'roles': ['ws_access']}, 'ws-example-1736870026': {'roles': ['ws_access']}, 'ws-example-79': {'roles': ['ws_access']}, 'ws-example-78': {'roles': ['ws_access']}, 'account': {'roles': ['manage-account', 'manage-account-links', 'view-profile']}}


### review existing workspaces

In [6]:
ws_user1 = [entry for entry in decoded_token_user1["aud"] if entry.startswith('ws-')]
print(f"user1: {ws_user1}")
ws_user2 = [entry for entry in decoded_token_user2["aud"] if entry.startswith('ws-')]
print(f"user2: {ws_user2}")

user1: ['ws-example-1736870119', 'ws-example-1736866203', 'ws-example-1736870026', 'ws-example-79', 'ws-example-78']
user2: []


In [7]:
def access_ws(ws_name, token):
    headers = {
        'Authorization': 'Bearer ' + token
    }
    url = f"{workspace_api_endpoint}/{ws_name}"
    print(f"HTTP GET {url}")
    response = requests.get(url, headers=headers)
    print(response)
    print(response.text)
    return response

In [8]:
#manually add/remove users in keycloak and check access here
access_ws("ws-example-78", token_user1)

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-78
<Response [200]>
{"status":"ready","endpoints":[],"storage":{"credentials":{"AWS_ACCESS_KEY_ID":"ws-example-78","AWS_ENDPOINT_URL":"https://minio.develop.eoepca.org","AWS_REGION":"eoepca-demo","AWS_SECRET_ACCESS_KEY":"IBbyCDqxxlteTpZcZkV3sZSDzZBsAmXbFuYfczV5RYHSPeLPBv9MKClpsv2TBrF0","access":"ws-example-78","bucketname":"ws-example-78","secret":"IBbyCDqxxlteTpZcZkV3sZSDzZBsAmXbFuYfczV5RYHSPeLPBv9MKClpsv2TBrF0","endpoint":"https://minio.develop.eoepca.org","region":"eu-central-1"}},"container_registry":null}


<Response [200]>

### workspace creation

In [9]:
preferred_ws_name = f"example-{datetime.datetime.now().timestamp():.0f}"
preferred_ws_name

'example-1736871929'

In [10]:
response = requests.post(
    workspace_api_endpoint,
    headers={
        #'Authorization': 'Bearer ' + token_user1
        'Authorization': 'Bearer ' + token_admin
    },
    json={
        "preferred_name": preferred_ws_name,
        "default_owner": "example-user-1"
    }
)
response.raise_for_status()
actual_workspace_name = response.json()['name']
ws_name = response.json()["name"]
print(f"created workspace '{ws_name}'")

created workspace 'ws-example-1736871929'


In [11]:
while True:
    time.sleep(3)
    response = access_ws(ws_name, iam_token("example-user-1", "changeme"))
    if response.status_code == 200:
        try:
            workspace_data = response.json()
            if workspace_data.get("status") == "ready":
                print("ready")
                break
        except ValueError:
            print("not ready yet")
    
    print("...retrying")
    

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736871929
<Response [403]>
<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>openresty</center>
<p><em>Powered by <a href="https://apisix.apache.org/">APISIX</a>.</em></p></body>
</html>

...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736871929
<Response [200]>
{"status":"provisioning","endpoints":[],"storage":null,"container_registry":null}
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736871929
<Response [200]>
{"status":"provisioning","endpoints":[],"storage":null,"container_registry":null}
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1736871929
<Response [200]>
{"status":"provisioning","endpoints":[],"storage":null,"container_registry":null}
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-exa

In [12]:
bucket_name = workspace_data["storage"]["credentials"]["bucketname"]
s3_access = workspace_data["storage"]["credentials"]["access"]
s3_secret = workspace_data["storage"]["credentials"]["secret"]

### workspace s3 access 

In [13]:
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=minio_endpoint)

In [14]:
path = 'application-package/s-expression/s-expression-0_0_2.cwl'

In [15]:
object = s3resource.Object(bucket_name, path)
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

Application package uploaded successfully


In [16]:
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(f"Key: {obj.key}, Size: {obj.size} bytes")

Key: application-package/s-expression/s-expression-0_0_2.cwl, Size: 2302 bytes


### workspace ui with storage layer

In [17]:
#after full IAM integration there will be no separate token here!
def ws_token(ws_name):
    url = f"https://{ws_name}.{base_domain}/api/login"
    print(f"HTTP GET {url}")
    response = requests.get(url, json={
        'username': 'default',
        'password': 'changeme',
    }, verify=False) # certificate creation ongoing...
    return response

def ws_share(path, token):
    url = f"https://{ws_name}.{base_domain}/api/share/sources/{bucket_name}/{path}"
    print(f"HTTP POST {url}")
    response = requests.post(url, headers={
        'x-auth': token
    }, json={}, verify=False) # certificate creation ongoing...
    return response

def ws_resolve(hash, follow):    
    url = f"https://{ws_name}.{base_domain}/api/public/dl/{hash}"
    if follow:
      url += "?inline=true"
    print(f"HTTP GET {url}")
    return requests.get(url, verify=False) # certificate creation ongoing...
    return response

In [18]:
while True:
    time.sleep(5)
    response = ws_token(ws_name)
    print(response)
    if response.ok:
        token=response.text
        break
    
    print("...retrying")

HTTP GET https://ws-example-1736871929.apx.develop.eoepca.org/api/login
<Response [200]>


In [19]:
from IPython.display import IFrame, display, HTML
display(HTML(f'<iframe src="https://{ws_name}.{base_domain}" width="100%" height="600"></iframe>'))

/home/achtsnits/github/eoepca/demo/.venv/lib/python3.12/site-packages/IPython/core/display.py:475: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [20]:
#create share link
share_response = ws_share(path, token)
print(share_response)
print(share_response.text)

HTTP POST https://ws-example-1736871929.apx.develop.eoepca.org/api/share/sources/ws-example-1736871929/application-package/s-expression/s-expression-0_0_2.cwl
<Response [200]>
{"hash":"KjtmA0Bu","path":"/sources/ws-example-1736871929/application-package/s-expression/s-expression-0_0_2.cwl","userID":1,"expire":0,"creationTime":1736871971}


In [21]:
#resolve public share link (no follow redirect)
while True:
    time.sleep(5)
    resolve_response = ws_resolve(share_response.json()["hash"], False)
    print(resolve_response)
    if resolve_response.ok:
        break
    
    print("...retrying")

HTTP GET https://ws-example-1736871929.apx.develop.eoepca.org/api/public/dl/KjtmA0Bu
<Response [404]>
...retrying
HTTP GET https://ws-example-1736871929.apx.develop.eoepca.org/api/public/dl/KjtmA0Bu
<Response [200]>


In [22]:
#resolve public share link (follow redirect)
resolve_response2 = ws_resolve(share_response.json()["hash"], True)
print(resolve_response2)
print(resolve_response2.text)

HTTP GET https://ws-example-1736871929.apx.develop.eoepca.org/api/public/dl/KjtmA0Bu?inline=true
<Response [200]>
$graph:
- baseCommand: s-expression
  class: CommandLineTool
  hints:
    DockerRequirement:
      dockerPull: eoepca/s-expression:dev0.0.2
  id: clt
  inputs:
    input_reference:
      inputBinding:
        position: 1
        prefix: --input_reference
      type: Directory
    s_expression:
      inputBinding:
        position: 2
        prefix: --s-expression
      type: string
    cbn:
      inputBinding:
        position: 3
        prefix: --cbn
      type: string
  outputs:
    results:
      outputBinding:
        glob: .
      type: Directory
  requirements:
    EnvVarRequirement:
      envDef:
        PATH: /srv/conda/envs/env_app_snuggs/bin:/srv/conda/envs/env_app_snuggs/bin:/srv/conda/bin:/srv/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
    ResourceRequirement: {}
  #stderr: std.err
  #stdout: std.out
- class: Workflow
  doc: Applies s

In [23]:
bucket.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '181A9BDD934C5964',
   'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Tue, 14 Jan 2025 16:26:22 GMT',
    'content-type': 'application/xml',
    'content-length': '201',
    'connection': 'keep-alive',
    'accept-ranges': 'bytes',
    'content-security-policy': 'block-all-mixed-content',
    'strict-transport-security': 'max-age=15724800; includeSubDomains',
    'vary': 'Origin, Accept-Encoding',
    'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
    'x-amz-request-id': '181A9BDD934C5964',
    'x-content-type-options': 'nosniff',
    'x-xss-protection': '1; mode=block'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'application-package/s-expression/s-expression-0_0_2.cwl'}]}]

In [24]:
#resolve public share link not possible anymore (404)
resolve_response3 = ws_resolve(share_response.json()["hash"], True)
print(resolve_response3)
print(resolve_response3.text)

HTTP GET https://ws-example-1736871929.apx.develop.eoepca.org/api/public/dl/KjtmA0Bu?inline=true
<Response [404]>
<?xml version="1.0" encoding="UTF-8"?>
<Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Key>application-package/s-expression/s-expression-0_0_2.cwl</Key><BucketName>ws-example-1736871929</BucketName><Resource>/ws-example-1736871929/application-package/s-expression/s-expression-0_0_2.cwl</Resource><RequestId>181A9BDDAE468670</RequestId><HostId>1eee01f8-c472-404f-ae10-a254b7dc7b08</HostId></Error>


### workspace deletion

In [25]:
response = requests.delete(
    f"{workspace_api_endpoint}/{ws_name}",
    headers={
        'Authorization': 'Bearer ' + iam_token("example-admin", "changeme")
    }
)
response.raise_for_status()
print(f"deleted workspace '{ws_name}'")

deleted workspace 'ws-example-1736871929'
